In [16]:
import sqlite3
import pandas as pd
import numpy as np

In [3]:
conn = sqlite3.connect("bluebikes.db")

In [4]:
stations = pd.read_sql_query("SELECT * FROM stations", conn)
rides = pd.read_sql_query("SELECT * FROM rides", conn)


In [5]:
stations.head()

,id,station_name
0,None,None
1,None,87-101 Cambridgepark Drive
2,None,Boylston St at Berkeley St
3,None,Boylston St at Exeter St
4,None,Boylston St at Fairfield St


In [6]:
rides.head()

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual
0,0CB95C5072A9A0E5,classic_bike,2025-04-13 21:06:54.496,2025-04-13 21:17:17.456,B32027,E32003,member
1,5C425780D5F59109,classic_bike,2025-04-28 12:38:52.585,2025-04-28 12:41:22.185,A32044,A32049,member
2,76808A97675DCECE,classic_bike,2025-04-14 11:36:20.428,2025-04-14 11:58:38.629,S32003,C32004,member
3,00E6835C21328359,classic_bike,2025-04-14 18:06:25.319,2025-04-14 18:21:04.129,M32053,B32058,member
4,484F5E541E4A4AD7,classic_bike,2025-04-24 18:32:17.408,2025-04-24 18:52:44.968,M32053,B32058,member


In [14]:
rides["ended_at_ts"] = pd.to_datetime(rides["ended_at"])
rides["started_at_ts"] = pd.to_datetime(rides["started_at"])

rides["length"] = rides["ended_at_ts"] - rides["started_at_ts"]
rides["length"] = rides["length"]/pd.Timedelta(1,'minute')
rides.head()

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual,ended_at_ts,started_at_ts,length
0,0CB95C5072A9A0E5,classic_bike,2025-04-13 21:06:54.496,2025-04-13 21:17:17.456,B32027,E32003,member,2025-04-13 21:17:17.456,2025-04-13 21:06:54.496,10.382667
1,5C425780D5F59109,classic_bike,2025-04-28 12:38:52.585,2025-04-28 12:41:22.185,A32044,A32049,member,2025-04-28 12:41:22.185,2025-04-28 12:38:52.585,2.493333
2,76808A97675DCECE,classic_bike,2025-04-14 11:36:20.428,2025-04-14 11:58:38.629,S32003,C32004,member,2025-04-14 11:58:38.629,2025-04-14 11:36:20.428,22.303350
3,00E6835C21328359,classic_bike,2025-04-14 18:06:25.319,2025-04-14 18:21:04.129,M32053,B32058,member,2025-04-14 18:21:04.129,2025-04-14 18:06:25.319,14.646833
4,484F5E541E4A4AD7,classic_bike,2025-04-24 18:32:17.408,2025-04-24 18:52:44.968,M32053,B32058,member,2025-04-24 18:52:44.968,2025-04-24 18:32:17.408,20.459333


In [ ]:
same_station_rides = rides[rides["start_station_id"]==rides["end_station_id"]]
avg_same_station_length = np.average(same_station_rides["length"])
avg_same_station_length # almost 34 minutes
len(same_station_rides)/len(rides) * 100.0 ## 3.2 percent

np.float64(33.9689066072662)

In [37]:
rides.groupby("member_casual").size()
rides.groupby("rideable_type").size()

rideable_type
classic_bike     261916
electric_bike    112024
dtype: int64

In [13]:
rides["ended_at_ts"] = pd.to_datetime(rides["ended_at"])
rides["started_at_ts"] = pd.to_datetime(rides["started_at"])

rides["length"] = rides["ended_at_ts"] - rides["started_at_ts"]
rides["length"] = rides["length"]/pd.Timedelta(1,'minute')
rides.head()

,ride_id,rideable_type,started_at,ended_at,start_station_id,end_station_id,member_casual,ended_at_ts,started_at_ts,length
0,0CB95C5072A9A0E5,classic_bike,2025-04-13 21:06:54.496,2025-04-13 21:17:17.456,B32027,E32003,member,2025-04-13 21:17:17.456,2025-04-13 21:06:54.496,10.382667
1,5C425780D5F59109,classic_bike,2025-04-28 12:38:52.585,2025-04-28 12:41:22.185,A32044,A32049,member,2025-04-28 12:41:22.185,2025-04-28 12:38:52.585,2.493333
2,76808A97675DCECE,classic_bike,2025-04-14 11:36:20.428,2025-04-14 11:58:38.629,S32003,C32004,member,2025-04-14 11:58:38.629,2025-04-14 11:36:20.428,22.303350
3,00E6835C21328359,classic_bike,2025-04-14 18:06:25.319,2025-04-14 18:21:04.129,M32053,B32058,member,2025-04-14 18:21:04.129,2025-04-14 18:06:25.319,14.646833
4,484F5E541E4A4AD7,classic_bike,2025-04-24 18:32:17.408,2025-04-24 18:52:44.968,M32053,B32058,member,2025-04-24 18:52:44.968,2025-04-24 18:32:17.408,20.459333
